In [13]:
"""
checking the pose estimation network and the loss to get the more better understanding of the architecture
and the losses and still have to write the 3d geometrical loss in temporal loss
"""

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Conv2DTranspose,UpSampling2D,MaxPooling2D,Concatenate,BatchNormalization,Flatten
from tensorflow.keras.layers import Dense,Layer
import numpy as np
from losses import *
from nets.dep_network import depth_estimation
from nets.pose_network import pose_est
from data_loader import data_loader_with_batch
#from utils import *

In [14]:
def euler2mat(z, y, x):
    """Converts euler angles to rotation matrix
    TODO: remove the dimension for 'N' (deprecated for converting all source
         poses altogether)
    Reference: https://github.com/pulkitag/pycaffe-utils/blob/master/rot_utils.py#L174
    Args:
      z: rotation angle along z axis (in radians) -- size = [B, N]
      y: rotation angle along y axis (in radians) -- size = [B, N]
      x: rotation angle along x axis (in radians) -- size = [B, N]
    Returns:
      Rotation matrix corresponding to the euler angles -- size = [B, N, 3, 3]
    """
    B = tf.shape(z)[0]
    N = 1
    z = tf.clip_by_value(z, -np.pi, np.pi)
    y = tf.clip_by_value(y, -np.pi, np.pi)
    x = tf.clip_by_value(x, -np.pi, np.pi)

    # Expand to B x N x 1 x 1
    z = tf.expand_dims(tf.expand_dims(z, -1), -1)
    y = tf.expand_dims(tf.expand_dims(y, -1), -1)
    x = tf.expand_dims(tf.expand_dims(x, -1), -1)

    zeros = tf.zeros([B, N, 1, 1])
    ones  = tf.ones([B, N, 1, 1])

    cosz = tf.cos(z)
    sinz = tf.sin(z)
    rotz_1 = tf.concat([cosz, -sinz, zeros], axis=3)
    rotz_2 = tf.concat([sinz,  cosz, zeros], axis=3)
    rotz_3 = tf.concat([zeros, zeros, ones], axis=3)
    zmat = tf.concat([rotz_1, rotz_2, rotz_3], axis=2)

    cosy = tf.cos(y)
    siny = tf.sin(y)
    roty_1 = tf.concat([cosy, zeros, siny], axis=3)
    roty_2 = tf.concat([zeros, ones, zeros], axis=3)
    roty_3 = tf.concat([-siny,zeros, cosy], axis=3)
    ymat = tf.concat([roty_1, roty_2, roty_3], axis=2)

    cosx = tf.cos(x)
    sinx = tf.sin(x)
    rotx_1 = tf.concat([ones, zeros, zeros], axis=3)
    rotx_2 = tf.concat([zeros, cosx, -sinx], axis=3)
    rotx_3 = tf.concat([zeros, sinx, cosx], axis=3)
    xmat = tf.concat([rotx_1, rotx_2, rotx_3], axis=2)

    rotMat = tf.matmul(tf.matmul(xmat, ymat), zmat)
    return rotMat

def pose_vec2mat(translation,rotation):
    """Converts 6DoF parameters to transformation matrix
    Args:
      vec: 6DoF parameters in the order of tx, ty, tz, rx, ry, rz -- [B, 6]
    Returns:
      A transformation matrix -- [B, 4, 4]
    """
    batch_size, _ = translation.get_shape().as_list()
    translation = tf.expand_dims(translation, -1)
    rx = tf.slice(rotation, [0, 0], [-1, 1])
    ry = tf.slice(rotation, [0, 1], [-1, 1])
    rz = tf.slice(rotation, [0, 2], [-1, 1])
    rot_mat = euler2mat(rz, ry, rx)
    rot_mat = tf.squeeze(rot_mat, axis=[1])
    filler = tf.constant([0.0, 0.0, 0.0, 1.0], shape=[1, 1, 4])
    filler = tf.tile(filler, [batch_size, 1, 1])
    transform_mat = tf.concat([rot_mat, translation], axis=2)
    transform_mat = tf.concat([transform_mat, filler], axis=1)
    return transform_mat

def pose_inv_vec2mat(translation,rotation):
    """Converts 6DoF parameters to transformation matrix
    Args:
      vec: 6DoF parameters in the order of tx, ty, tz, rx, ry, rz -- [B, 6]
    Returns:
      A transformation matrix -- [B, 4, 4]
    """
    batch_size, _ = translation.get_shape().as_list()
    translation = tf.expand_dims(translation, -1)
    rx = tf.slice(rotation, [0, 0], [-1, 1])
    ry = tf.slice(rotation, [0, 1], [-1, 1])
    rz = tf.slice(rotation, [0, 2], [-1, 1])
    rot_mat = euler2mat(rz, ry, rx)
    rot_mat_transpose = tf.transpose(tf.squeeze(rot_mat, axis=[1]))
    filler = tf.constant([0.0, 0.0, 0.0, 1.0], shape=[1, 1, 4])
    filler = tf.tile(filler, [batch_size, 1, 1])
    inv_homo_trans = -tf.matmul(rot_mat_transpose,translation)
    transform_mat = tf.concat([rot_mat, inv_homo_trans], axis=2)
    transform_mat = tf.concat([transform_mat, filler], axis=1)
    return transform_mat

def pixel2cam(depth, pixel_coords, intrinsics, is_homogeneous=True):
    """Transforms coordinates in the pixel frame to the camera frame.
    Args:
    depth: [batch, height, width]
    pixel_coords: homogeneous pixel coordinates [batch, 3, height, width]
    intrinsics: camera intrinsics [batch, 3, 3]
    is_homogeneous: return in homogeneous coordinates
    Returns:
    Coords in the camera frame [batch, 3 (4 if homogeneous), height, width]
    """
    batch, height, width = depth.get_shape().as_list()
    depth = tf.reshape(depth, [batch, 1, -1])
    pixel_coords = tf.reshape(pixel_coords, [batch, 3, -1])
    cam_coords = tf.matmul(tf.linalg.inv(intrinsics), pixel_coords) * depth
    if is_homogeneous:
        ones = tf.ones([batch, 1, height*width])
        cam_coords = tf.concat([cam_coords, ones], axis=1)
    cam_coords = tf.reshape(cam_coords, [batch, -1, height, width])
    return cam_coords

def cam2pixel(cam_coords, proj):
    """Transforms coordinates in a camera frame to the pixel frame.
    Args:
    cam_coords: [batch, 4, height, width]
    proj: [batch, 4, 4]
    Returns:
    Pixel coordinates projected from the camera frame [batch, height, width, 2]
    """
    batch, _, height, width = cam_coords.get_shape().as_list()
    cam_coords = tf.reshape(cam_coords, [batch, 4, -1])
    unnormalized_pixel_coords = tf.matmul(proj, cam_coords)
    x_u = tf.slice(unnormalized_pixel_coords, [0, 0, 0], [-1, 1, -1])
    y_u = tf.slice(unnormalized_pixel_coords, [0, 1, 0], [-1, 1, -1])
    z_u = tf.slice(unnormalized_pixel_coords, [0, 2, 0], [-1, 1, -1])
    x_n = x_u / (z_u + 1e-10)
    y_n = y_u / (z_u + 1e-10)
    pixel_coords = tf.concat([x_n, y_n], axis=1)
    pixel_coords = tf.reshape(pixel_coords, [batch, 2, height, width])
    return tf.transpose(pixel_coords, perm=[0, 2, 3, 1])

def meshgrid(batch, height, width, is_homogeneous=True):        
    """Construct a 2D meshgrid.
    Args:
    batch: batch size
    height: height of the grid
    width: width of the grid
    is_homogeneous: whether to return in homogeneous coordinates
    Returns:
    x,y grid coordinates [batch, 2 (3 if homogeneous), height, width]
    """
    """
    x_t = tf.matmul(tf.ones(shape=tf.stack([height, 1])),
                  tf.transpose(tf.expand_dims(
                      tf.linspace(-1.0, 1.0, width), 1), [1, 0]))
    y_t = tf.matmul(tf.expand_dims(tf.linspace(-1.0, 1.0, height), 1),
                  tf.ones(shape=tf.stack([1, width])))
    x_t = (x_t + 1.0) * 0.5 * tf.cast(width - 1, tf.float32)
    y_t = (y_t + 1.0) * 0.5 * tf.cast(height - 1, tf.float32)
    """
    width_f = tf.cast(width,tf.float32)
    height_f = tf.cast(height,tf.float32)
    x_t,y_t = tf.meshgrid(tf.linspace(0.0,width_f,width),tf.linspace(0.0,height_f,height))
    if is_homogeneous:
        ones = tf.ones_like(x_t)
        coords = tf.stack([x_t, y_t, ones], axis=0)
    else:
        coords = tf.stack([x_t, y_t], axis=0)
    coords = tf.tile(tf.expand_dims(coords, 0), [batch, 1, 1, 1])
    return coords

def projective_inverse_warp(img, depth, translate,rotate, intrinsics,inv=False):
    """Inverse warp a source image to the target image plane based on projection.
    Args:
    img: the source image [batch, height_s, width_s, 3]
    depth: depth map of the target image [batch, height_t, width_t]
    pose: target to source camera transformation matrix [batch, 6], in the
          order of tx, ty, tz, rx, ry, rz
    intrinsics: camera intrinsics [batch, 3, 3]
    Returns:
    Source image inverse warped to the target image plane [batch, height_t,
    width_t, 3]
    """
    batch, height, width, _ = img.get_shape().as_list()
    # Convert pose vector to matrix
    pose = pose_vec2mat(translate,rotate)
    if inv==False:
        pose=pose
    else:
        pose = tf.linalg.inv(pose)
    # Construct pixel grid coordinates
    pixel_coords = meshgrid(batch, height, width)
    # Convert pixel coordinates to the camera frame
    cam_coords = pixel2cam(depth, pixel_coords, intrinsics)
    # Construct a 4x4 intrinsic matrix (TODO: can it be 3x4?)
    filler = tf.constant([0.0, 0.0, 0.0, 1.0], shape=[1, 1, 4])
    filler = tf.tile(filler, [batch, 1, 1])
    intrinsics = tf.concat([intrinsics, tf.zeros([batch, 3, 1])], axis=2)
    intrinsics = tf.concat([intrinsics, filler], axis=1)
    # Get a 4x4 transformation matrix from 'target' camera frame to 'source'
    # pixel frame.
    proj_tgt_cam_to_src_pixel = tf.matmul(intrinsics, pose)
    src_pixel_coords = cam2pixel(cam_coords, proj_tgt_cam_to_src_pixel)
    output_img = bilinear_sampler(img, src_pixel_coords)
    return output_img 
def bilinear_sampler(imgs, coords):
    """Construct a new image by bilinear sampling from the input image.
    Points falling outside the source image boundary have value 0.
    Args:
    imgs: source image to be sampled from [batch, height_s, width_s, channels]
    coords: coordinates of source pixels to sample from [batch, height_t,
      width_t, 2]. height_t/width_t correspond to the dimensions of the output
      image (don't need to be the same as height_s/width_s). The two channels
      correspond to x and y coordinates respectively.
    Returns:
    A new sampled image [batch, height_t, width_t, channels]
    """
    def _repeat(x, n_repeats):        
        rep = tf.transpose(
            tf.expand_dims(tf.ones(shape=tf.stack([
                n_repeats,
            ])), 1), [1, 0])
        rep = tf.cast(rep, 'float32')
        x = tf.matmul(tf.reshape(x, (-1, 1)), rep)
        return tf.reshape(x, [-1])


    coords_x, coords_y = tf.split(coords, [1, 1], axis=3)
    inp_size = imgs.get_shape()
    coord_size = coords.get_shape()
    out_size = coords.get_shape().as_list()
    out_size[3] = imgs.get_shape().as_list()[3]

    coords_x = tf.cast(coords_x, 'float32')
    coords_y = tf.cast(coords_y, 'float32')

    x0 = tf.floor(coords_x)
    x1 = x0 + 1
    y0 = tf.floor(coords_y)
    y1 = y0 + 1

    y_max = tf.cast(tf.shape(imgs)[1] - 1, 'float32')
    x_max = tf.cast(tf.shape(imgs)[2] - 1, 'float32')
    zero = tf.zeros([1], dtype='float32')

    x0_safe = tf.clip_by_value(x0, zero, x_max)
    y0_safe = tf.clip_by_value(y0, zero, y_max)
    x1_safe = tf.clip_by_value(x1, zero, x_max)
    y1_safe = tf.clip_by_value(y1, zero, y_max)

    ## bilinear interp weights, with points outside the grid having weight 0
    # wt_x0 = (x1 - coords_x) * tf.cast(tf.equal(x0, x0_safe), 'float32')
    # wt_x1 = (coords_x - x0) * tf.cast(tf.equal(x1, x1_safe), 'float32')
    # wt_y0 = (y1 - coords_y) * tf.cast(tf.equal(y0, y0_safe), 'float32')
    # wt_y1 = (coords_y - y0) * tf.cast(tf.equal(y1, y1_safe), 'float32')

    wt_x0 = x1_safe - coords_x
    wt_x1 = coords_x - x0_safe
    wt_y0 = y1_safe - coords_y
    wt_y1 = coords_y - y0_safe

    ## indices in the flat image to sample from
    dim2 = tf.cast(inp_size[2], 'float32')
    dim1 = tf.cast(inp_size[2] * inp_size[1], 'float32')
    base = tf.reshape(
        _repeat(
            tf.cast(tf.range(coord_size[0]), 'float32') * dim1,
            coord_size[1] * coord_size[2]),
        [out_size[0], out_size[1], out_size[2], 1])

    base_y0 = base + y0_safe * dim2
    base_y1 = base + y1_safe * dim2
    idx00 = tf.reshape(x0_safe + base_y0, [-1])
    idx01 = x0_safe + base_y1
    idx10 = x1_safe + base_y0
    idx11 = x1_safe + base_y1

    ## sample from imgs
    imgs_flat = tf.reshape(imgs, tf.stack([-1, inp_size[3]]))
    imgs_flat = tf.cast(imgs_flat, 'float32')
    im00 = tf.reshape(tf.gather(imgs_flat, tf.cast(idx00, 'int32')), out_size)
    im01 = tf.reshape(tf.gather(imgs_flat, tf.cast(idx01, 'int32')), out_size)
    im10 = tf.reshape(tf.gather(imgs_flat, tf.cast(idx10, 'int32')), out_size)
    im11 = tf.reshape(tf.gather(imgs_flat, tf.cast(idx11, 'int32')), out_size)

    w00 = wt_x0 * wt_y0
    w01 = wt_x0 * wt_y1
    w10 = wt_x1 * wt_y0
    w11 = wt_x1 * wt_y1

    output = tf.add_n([
        w00 * im00, w01 * im01,
        w10 * im10, w11 * im11
    ])
    return output


In [15]:
#pe = pose_est()
def pose_est_loss(rot_right,rot_left,t_left,t_right):
    ##Spatial loss
    #Pose consistency loss
    lambd_p = 0.5
    lambd_o = 0.5

    pc_loss = lambd_p*tf.reduce_mean(tf.abs(t_right-t_left))+lambd_o*tf.reduce_mean(tf.abs(rot_right-rot_left))

    #Temporal loss
    
    return pc_loss

def pose_temporal_loss(pk,pk1,translate,rotate,intrinsics,depth):
    """
    Argument :
    pk is the pixel in the current frame 
    pk1 which is p_k+1 is the next frame pixel or next image 
    """
    ##loss for predicting the pk+1
    lambd = 0.5
    pk1_pred = projective_inverse_warp(pk, depth, translate,rotate, intrinsics)
    ssim_loss_pk1 = SSIM(pk1,pk1_pred) 
    l1_loss_pk1 = tf.reduce_mean(tf.abs(tf.subtract(pk1_pred,pk1)))
    loss_pk1 = lambd*ssim_loss_pk1+(1-lambd)*l1_loss_pk1
    
    ##loss for predicting the pk
    lambd = 0.5
    pk_pred = projective_inverse_warp(pk, depth, translate,rotate, intrinsics,inv=True)
    ssim_loss_pk = SSIM(pk,pk_pred) 
    l1_loss_pk = tf.reduce_mean(tf.abs(tf.subtract(pk_pred,pk)))
    loss_pk = lambd*ssim_loss_pk+(1-lambd)*l1_loss_pk
    
    photo_temporal_loss = loss_pk+loss_pk1
    
    return photo_temporal_loss
    

In [25]:
optimizer = tf.keras.optimizers.Adam(learning_rate=.001)
batch=2        
epsilon=0.0000001
@tf.function
def train_step(image_left,image_right):
    with tf.GradientTape(persistent=True) as tape:
        intrinsic = tf.expand_dims(tf.constant([[1,2,3],[4,2,4],[2,1,6]],dtype=tf.float32),axis=0)
        intrinsic = tf.tile(intrinsic,[batch,1,1])

        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        dcx7_right,dep_right = de(image_right)
        #dcx7_left,dep_left = de(image_left)
        #loss_depth = dept_est_loss(dep_left,dep_right,image_left,image_right)
        #tf.print("loss_depth",loss_depth)
        ##not using pose estimation currently because of tensorflow 2.0
        t_left,rot_left = pe(image_left)
        t_right,rot_right = pe(image_right)
        loss_pose = pose_est_loss(rot_right,rot_left,t_left,t_right)
        disp = (dep_right[:,:,:,0]+epsilon)
        depth = 1/disp
        pk = tf.slice(image_right,[0,0,0,0],[-1,-1,-1,3])
        pk1 = tf.slice(image_right,[0,0,0,3],[-1,-1,-1,3])
        loss_pose_1 = pose_temporal_loss(pk,pk1,t_right,rot_right,intrinsic,depth)
        #tf.print("loss_pose",loss_pose)
        loss_pose_total = loss_pose+loss_pose_1

    #gradients_de = tape.gradient(loss_depth, de.trainable_variables)
    gradients_pe = tape.gradient(loss_pose_total, pe.trainable_variables)
    #optimizer.apply_gradients(zip(gradients_de, de.trainable_variables))
    optimizer.apply_gradients(zip(gradients_pe, pe.trainable_variables))
    return loss_pose

In [26]:
EPOCHS = 5
train_ds = []
train_ps = []
de = depth_estimation()
pe = pose_est()
"""
for i in range(5):
    #images_left = np.random.randint(0,255,(3,256,256,3)).astype(np.float32)
    #images_right = np.random.randint(0,255,(3,256,256,3*2)).astype(np.float32)
    images_left = tf.ones((3,128,416,3*2),dtype=tf.float32)*(i)
    images_right = tf.ones((3,128,416,3*2),dtype=tf.float32)*(i+2)
    train_ds.append([images_left,images_right])
for j in range(5):
    images_k = tf.ones((3,128,416,3*2))*(j)
    images_k1 = tf.ones((3,128,416,3*2))*(j+1)
    train_ps.append([images_k,images_k1])
"""
path_dir = "/home/roboticist/Documents/Swaayatt/swaayatt_optical_flow/dataset_undeepVO/data_scene_flow(1)/training"
left_img_dir = path_dir+"/image_2/"
right_img_dir = path_dir+"/image_3/"
img_width = 416
img_height = 128

train_ds = data_loader_with_batch(left_img_dir,right_img_dir,img_width,img_height,batch=2,number_of_data=50)

#de = depth_estimation()
for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    for images_left,images_right in train_ds:
        ld = train_step(images_left,images_right)

    print("epochs : ",epoch)
    print("loss depth",ld.numpy())
    #print("loss pose",lp)

epochs :  0
loss depth 4.3044718e-05
epochs :  1
loss depth 1.1760587e-05
epochs :  2
loss depth 3.912331e-06
epochs :  3
loss depth 3.0100346e-06
epochs :  4
loss depth 5.2812197e-06


In [ ]:
batch=3
intrinsic = tf.expand_dims(tf.constant([[1,1,1],[1,1,1],[1,1,1]],dtype=tf.float32),axis=0)
intrinsic = tf.tile(intrinsic,[batch,1,1])
#intrinsic.dtype
tf.linalg.inv(intrinsic)

In [24]:
_,dep_right = de(images_left)
disp = dep_right[:,:,:,0]+epsilon
depth = 1/disp
print(dep_right.shape)
print(disp.shape)
print(depth)

(2, 128, 416, 2)
(2, 128, 416)
tf.Tensor(
[[[1.9969864 2.002676  2.0044484 ... 2.0424032 2.0537748 2.0043957]
  [1.9953642 2.0000772 1.9995693 ... 1.9448109 1.9526278 1.9309225]
  [1.9914441 1.9981756 1.9991022 ... 2.0341833 2.1246343 2.101088 ]
  ...
  [1.9922311 1.9892225 2.001728  ... 1.9941452 1.9999864 1.9929854]
  [1.9975587 2.0160933 2.007829  ... 2.0019073 2.0081086 2.0005596]
  [1.9950733 1.9955968 1.9928772 ... 1.9982989 1.9952387 1.9978765]]

 [[1.9904242 2.0091746 2.0173743 ... 2.009306  2.0230606 2.0284717]
  [1.9934739 1.9966097 1.9996564 ... 1.9276782 1.9578804 1.9479724]
  [1.9800311 1.9849877 1.9986681 ... 2.002211  2.0506854 2.0601249]
  ...
  [2.0008576 1.9942001 1.9962764 ... 2.0428753 2.0220377 1.9823285]
  [1.9961455 2.0110211 2.0041008 ... 1.985572  2.0180545 1.9918451]
  [1.9974968 1.9965783 1.9934357 ... 1.9888284 1.9853305 1.9847068]]], shape=(2, 128, 416), dtype=float32)
